In [192]:
from IPython.core.display import display, HTML
display(HTML("<style>.container {width:100% !important;}<style>"))

# Cluster

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
import os
import json
from tqdm import tqdm
import random

%matplotlib inline
DATA_SHAPE = 224

val_labels = np.load('/home/mc16/pre_data/val_label_%s.npy'%DATA_SHAPE)
val_detect_points = np.load('/home/mc16/result/val_masks_06-11-18:34:58.npy')
test_detect_points = np.load('/home/mc16/result/test_masks_06-11-18:34:33.npy')

IOError: [Errno 2] No such file or directory: '/home/mc16/result/val_masks_06-11-18:34:58.npy'

In [3]:
# format matrix to point(x,y)
def matrix_to_point(detect_points):
    points_list = [[] for i in range(len(detect_points))]
    for i in tqdm(range(detect_points.shape[0])):
        for x in range(detect_points[i].shape[0]):
            for y in range(detect_points[i].shape[1]):
                if(detect_points[i][x][y] == 1):
                    points_list[i].append([x, y])
    return np.array(points_list)

val_point_data = matrix_to_point(val_detect_points)
test_point_data = matrix_to_point(test_detect_points)

NameError: name 'val_detect_points' is not defined

In [9]:
print(type(val_point_data[0][0]))

<type 'list'>


In [15]:
# cluster points to coarse lines
def get_cluster_points(points_list, EPS = 1.5, MIN_SAMPLES = 3):
    cluster_points_list = []
    for points in tqdm(points_list):
        if points == []: 
            cluster_points = []
        else:
            arpoints = np.array(points, dtype=np.uint8)
            cluster_label = DBSCAN(eps=EPS, min_samples=MIN_SAMPLES).fit_predict(arpoints)
            max_label = np.max(cluster_label)
            cluster_points = []
            for label in range(-1, max_label+1):
                label_index = np.where(cluster_label == label)
                cluster_points.append(arpoints[label_index])
        cluster_points_list.append(np.array(cluster_points))
    return np.array(cluster_points_list)

val_cluster_list = get_cluster_points(val_point_data, EPS = 1.5, MIN_SAMPLES = 3)
test_cluster_list = get_cluster_points(test_point_data, EPS = 1.5, MIN_SAMPLES = 3)

100%|██████████| 2029/2029 [00:19<00:00, 104.74it/s]


In [32]:
print(type(val_cluster_list), type(val_cluster_list[0]), type(val_cluster_list[0][0]), val_cluster_list[0][1].max())

(<type 'numpy.ndarray'>, <type 'numpy.ndarray'>, <type 'numpy.ndarray'>, 152)


In [1]:
def cluster_to_img(cluster_points, shape):
    pic = np.zeros((shape, shape), np.uint8)
    num_label = cluster_points.shape[0]
    for i in range(num_label):
        for point in cluster_points[i]:
            pic[point[0], point[1]] = i * int(255 / num_label)
    return pic

test_index = random.randint(0, len(clustered_point_list))
test_label = val_labels[test_index,:,:,1]
test_points = val_detect_points[test_index]
cluster_img = cluster_to_img(clustered_point_list[test_index], DATA_SHAPE)

fig, axs = plt.subplots(1, 3, figsize=(30,30)) 
axs[0].imshow(test_label,cmap='gray')
axs[1].imshow(test_points,cmap='gray')
axs[2].imshow(cluster_img,cmap='gray')
plt.show()

NameError: name 'random' is not defined

# Save result

In [205]:
import datetime
TIMEFORMAT = '%m-%d-%H:%M:%S'
time = datetime.datetime.now().strftime(TIMEFORMAT)
np.save("/home/mc16/result/test_cluster_%s.npy"%time, test_cluster_list)
time = datetime.datetime.now().strftime(TIMEFORMAT)
np.save("/home/mc16/result/val_cluster_%s.npy"%time, val_cluster_list)